<a href="https://colab.research.google.com/github/b08202011/Mlproject/blob/main/L2regulizer%2B%E6%9C%80%E7%B0%A1%E5%96%AE%E7%9A%84model%2Bval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown 1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
!gdown 15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
!gdown 187gijxd4T5yuZe_g2K9UNepx7MT1CnAu

Downloading...
From: https://drive.google.com/uc?id=1lpwTva7SEtLRNwpBbYKZLoHDoeG7FTxp
To: /content/train.csv
100% 23.3M/23.3M [00:00<00:00, 150MB/s]
Downloading...
From: https://drive.google.com/uc?id=15MfSdxpfkHQi_Z0qdJIdrdPzLrSfbX1E
To: /content/test.csv
100% 8.88M/8.88M [00:00<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=187gijxd4T5yuZe_g2K9UNepx7MT1CnAu
To: /content/sample_submission.csv
100% 50.5k/50.5k [00:00<00:00, 133MB/s]


In [ ]:
import csv
import numpy as np
from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(missing_values=np.nan, strategy='median')
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')


In [ ]:
!pip install ycimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import csv
import pandas as pd
import copy
import math
from ycimpute.imputer import knnimput
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier


def valid(test_y, y):
    Ein = 0
    for i in range(len(y)):
        Ein = Ein + abs(int(float(y[i]))-int(float(test_y[i])))
    return (Ein/len(y))


def normalize_knn(X: np.ndarray) -> np.ndarray:
    mean = np.nanmean(X, axis=0)
    std = np.nanstd(X, axis=0)
    for i in range(len(X)):
        for j in range(len(X[i])):
            if(X[i][j] != np.nan):
                X[i][j] = (X[i][j]-mean[j])/std[j]
    X = np.array(X)
    X = knnimput.KNN(k=15).complete(X)
    return X


def com(feature, composer, data):

    for i in range(1, len(data)):
        for j in range(len(composer)-1):
            feature[i-1].append(0)
    for i in range(1, len(data)):
        one_hot = composer.index(data[i][-2])
        if(one_hot != 0):
            feature[i-1][-one_hot] = 1
        else:
            for j in range(1, len(composer)):
                feature[i-1][-j] = np.nan
    return feature


with open('train.csv', newline='', encoding="utf-8") as csvfile:
    train = csv.reader(csvfile)
    train = list(train)

composer = []
for i in range(1, len(train)):
    if(train[i][-2] not in composer):
        composer.append(train[i][-2])

pick = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 23]

feature = []
for i in range(1, len(train)):
    feature.append([])
    for j in pick:
        feature[i-1].append(train[i][j])
for i in range(len(feature)):
    for j in range(len(feature[i])):
        if(feature[i][j] != '' and feature[i][j] != 'False' and feature[i][j] != 'True'):
            feature[i][j] = float(feature[i][j])
        elif(feature[i][j] == 'False'):
            feature[i][j] = 0
        elif(feature[i][j] == 'True'):
            feature[i][j] = 1
        else:
            feature[i][j] = np.nan

feature = com(feature, composer, train)

for i in range(1, len(train)):
    if(train[i][14] == 'compilation'):
        feature[i-1].append(1)
        feature[i-1].append(0)
        feature[i-1].append(0)
    elif(train[i][14] == 'album'):
        feature[i-1].append(0)
        feature[i-1].append(1)
        feature[i-1].append(0)
    elif(train[i][14] == 'single'):
        feature[i-1].append(0)
        feature[i-1].append(0)
        feature[i-1].append(1)
    else:
        feature[i-1].append(np.nan)
        feature[i-1].append(np.nan)
        feature[i-1].append(np.nan)

y_train = []
for i in range(1, len(train)):
    y_train.append(int(float(train[i][0])))


with open('test.csv', newline='', encoding="utf-8") as csvfile:
    test = csv.reader(csvfile)
    test = list(test)

pick = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 22]

# 可以選要用那些特徵，0是我們要predict的
# 只選數值或true fale 但且沒有選id
feature_test = []
for i in range(1, len(test)):
    feature_test.append([])
    for j in pick:
        feature_test[i-1].append(test[i][j])

for i in range(len(feature_test)):
    for j in range(len(feature_test[i])):
        if(feature_test[i][j] != '' and feature_test[i][j] != 'False' and feature_test[i][j] != 'True'):
            feature_test[i][j] = float(feature_test[i][j])
        elif(feature_test[i][j] == 'False'):
            feature_test[i][j] = 0
        elif(feature_test[i][j] == 'True'):
            feature_test[i][j] = 1
        else:
            feature_test[i][j] = np.nan
feature_test = com(feature_test, composer, test)

for i in range(1, len(test)):
    if(test[i][13] == 'compilation'):
        feature_test[i-1].append(1)
        feature_test[i-1].append(0)
        feature_test[i-1].append(0)
    elif(test[i][13] == 'album'):
        feature_test[i-1].append(0)
        feature_test[i-1].append(1)
        feature_test[i-1].append(0)
    elif(test[i][13] == 'single'):
        feature_test[i-1].append(0)
        feature_test[i-1].append(0)
        feature_test[i-1].append(1)
    else:
        feature_test[i-1].append(np.nan)
        feature_test[i-1].append(np.nan)
        feature_test[i-1].append(np.nan)


feature = feature + feature_test


feature = np.array(feature)

feature = normalize_knn(feature)
feature_train = feature[0:len(train)-1]





Imputing row 1/23485 with 11 missing, elapsed time: 185.149
Imputing row 101/23485 with 1 missing, elapsed time: 185.191
Imputing row 201/23485 with 0 missing, elapsed time: 185.235
Imputing row 301/23485 with 2 missing, elapsed time: 185.281
Imputing row 401/23485 with 1 missing, elapsed time: 185.327
Imputing row 501/23485 with 1 missing, elapsed time: 185.379
Imputing row 601/23485 with 4 missing, elapsed time: 185.431
Imputing row 701/23485 with 5 missing, elapsed time: 185.487
Imputing row 801/23485 with 2 missing, elapsed time: 185.547
Imputing row 901/23485 with 13 missing, elapsed time: 185.596
Imputing row 1001/23485 with 0 missing, elapsed time: 185.654
Imputing row 1101/23485 with 0 missing, elapsed time: 185.701
Imputing row 1201/23485 with 2 missing, elapsed time: 185.753
Imputing row 1301/23485 with 3 missing, elapsed time: 185.811
Imputing row 1401/23485 with 3 missing, elapsed time: 185.859
Imputing row 1501/23485 with 3 missing, elapsed time: 185.914
Imputing row 1601/

In [ ]:
test_data = np.array(feature[len(train)-1:len(feature)])
print(test_data.shape)
ytrain = np.array(y_train)
ytrain = np.reshape(ytrain,(ytrain.shape[0],1))
feature = np.concatenate((ytrain,np.array(feature_train)),axis=1)

(6315, 29)


In [ ]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv
from sklearn.feature_selection import SelectKBest
# For Progress Bar
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.feature_selection import RFE
from sklearn.feature_selection import f_regression
# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork
from sklearn.feature_selection import VarianceThreshold
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import utils

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter
from torch.nn.functional import normalize

In [ ]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = torch.clamp(model(x),0,9)
            #pred = np.array(pred.tolist())
            #predict = np.zeros((pred.shape[0]))
            #for i in range(pred.shape[0]):
            #  rate = 0
            #  for j in range(pred.shape[1]):
            #    rate += pred[i][j]
            #    if rate >0.5:
            #      predict[i] = j 
            #      break
            #pred = torch.from_numpy(predict).to(device)              
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

##Dataloader

In [ ]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

## Model

In [ ]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, 16),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
                                     
        )
        self.maxpoollayer = nn.Sequential(
            nn.BatchNorm1d(16),
            nn.Dropout(0.1),
            nn.Linear(16, 64),
            nn.BatchNorm1d(64),
            nn.Sigmoid(),
            nn.MaxPool1d(kernel_size=4),
            nn.BatchNorm1d(16),
        )
        self.finallayer2 = nn.Sequential(
            nn.BatchNorm1d(16),
            nn.Tanh(),                  ## model layer 可以隨便調，不知道哪裡有現成的可以找
            nn.Dropout(0.2),
            nn.Linear(16, 10),
        )
        
        self.finallayer = nn.Sequential(
            nn.BatchNorm1d(16),
            nn.Linear(16, 640),
            nn.BatchNorm1d(640),
            nn.Tanh(),          
            nn.Linear(640, 1000),
            nn.BatchNorm1d(1000),
            nn.Tanh(),
            nn.Dropout(0.2),
            nn.Linear(1000, 10),
        )

        self.softmax = nn.Softmax(dim=1)
        self.fc = nn.Linear(10, 1)

        self.rnn = nn.RNN(input_dim, 256, 3)
        self.fc2 = nn.Linear(256, 10)

        
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        #RNN
        #h0 = torch.randn(3, 256).to(device)             
        #outRnn, _ = self.rnn(x, h0)
        
        #outRnn = self.fc2(outRnn)
        #outRnn = self.softmax(outRnn)
        z = self.layers(x)
        #z = self.layers2(z)

 
        #y = self.layers(x)
        #x = self.layers(x)
        #x = self.maxpoollayer(x)
        #x = self.maxpoollayer2(x)
        #x = self.maxpoollayer3(x)
        #x = self.finallayer(x)
        #x = self.softmax(x)
        #y = self.finallayer(y)
        # = self.softmax(y)
        


        #x = torch.add(torch.mul(x,0.7),torch.mul(y,0.3))
        #x = torch.add(torch.mul(x,0.9),torch.mul(outRnn,0.1))   #把不同model混在一起的

        #out = torch.linspace(0,9,10).to(device)           ##算期望值的 如果輸出是0...9的機率
        #x = torch.mul(x,out)
        #x = torch.sum(x, dim=1)                  
                               
#        y = np.array(y.tolist())                  ###算中位數的
#        temp = np.zeros((y.shape[0]))
#        for i in range(y.shape[0]):
#          rate = 0
#          for j in range(y.shape[1]):
#           rate += y[i][j]
#            if rate >0.5:
#              temp[i] = j 
#              break
#        y = torch.from_numpy(temp).to(device)

        z = self.finallayer(z)
        #z = self.sigmoid(z)                    ##拿掉可能比較好一點點?但沒差太多
        z = self.fc(z)                      ###z 
        #output = torch.add(torch.mul(x,0.0001),torch.mul(z,0.9999))
        #
        #z = torch.mul(z,9)
        #x = self.fc(x)
        output = z
        #
        
        output = output.squeeze(-1) # (B, 1) -> (B)
        return output
class wideL1loss(nn.Module):                     #####區間的loss
  def __init__(self):
    super(wideL1loss, self).__init__()
  def forward(self, x,y):
    loss = torch.mean(torch.subtract(torch.clamp(torch.abs(x-y),2.5,10),2.5))
    return loss


## Training

In [ ]:
def trainer(train_data, model, config, device):

    criterion = nn.SmoothL1Loss(beta=0.5) # Define your loss function, do not modify this.
    #criterion = nn.CrossEntropyLoss() 
    loss2 = wideL1loss()                  ###我用來算區間的loss
    criterion2 = nn.L1Loss() 

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.AdamW(model.parameters(), lr= config['learning_rate'] , weight_decay=0.01, amsgrad=False) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0


    np.random.seed(config['seed'])
    splits = np.array_split(train_data, 5)
    train =  np.concatenate(splits[:0]+splits[0+1:])
    valid= splits[0]
    x_train = train[...,1:]                          ###現在沒用cross validation
    x_valid = valid[...,1:]
    y_train = train[...,0]
    y_valid = valid[...,0]
    train_dataset  = COVID19Dataset(x_train, y_train)
    valid_dataset = COVID19Dataset(x_valid, y_valid)
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
    epoachrecord = 0
    for epoch in range(n_epochs):
      lossvalid_record = []
      loss_record = []
      realloss = []
      for i in range(1):
        model.train() # Set your model to train mode.
         
        
        # Pytorch data loader loads pytorch dataset into batches.
        
        for x, y in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)
            

            
            #loss = criterion(pred, y)
            l2_norm = sum((p.pow(2)/(1+p.pow(2))).sum()for p in model.parameters())        ##上課的l2 norm resize 過的不知道加個log會不會更好
            loss = criterion(pred, y)+config['regulizer']*l2_norm+config['loss2']*loss2(pred, y)
            #print(loss)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            
            loss_record.append(loss.detach().item())
 
        model.eval() # Set your model to evaluation mode.
        
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = torch.clamp(model(x),0,9)
                #loss = criterion(pred, y)
                l2_norm = sum((p.pow(2)/(1+p.pow(2))).sum()for p in model.parameters())
                loss = criterion(pred, y)+config['regulizer']*l2_norm+config['loss2']*loss2(pred, y)
                realloss2 = criterion2(pred, y)
            lossvalid_record.append(loss.item())
            realloss.append(realloss2.item())

        
      mean_train_loss = sum(loss_record)/len(loss_record)         ###compute CV loss
      writer.add_scalar('Loss/train', mean_train_loss, step)
      reloss = sum(realloss)/len(realloss)
      mean_valid_loss = sum(lossvalid_record)/len(lossvalid_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, L1 loss: {reloss:.4f}  ', epoachrecord)
      # writer.add_scalar('Loss/valid', mean_valid_loss, step)

      if mean_valid_loss < best_loss:
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
          epoachrecord = epoch
      else: 
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return
    


In [ ]:
def trainer2(train_data, model, epoachrecord,config, device):

    criterion = nn.SmoothL1Loss(beta=0.5) # Define your loss function, do not modify this.
    #criterion = nn.CrossEntropyLoss() 
    loss2 = wideL1loss()                  ###我用來算區間的loss
    criterion2 = nn.L1Loss() 

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    # TODO: L2 regularization (optimizer(weight decay...) or implement by your self).
    optimizer = torch.optim.AdamW(model.parameters(), lr= config['learning_rate'] , weight_decay=0.01, amsgrad=False) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
    x_train = train_data[...,1:]
    y_train = train_data[...,0]
    train_dataset  = COVID19Dataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
    for epoch in range(epoachrecord):
      model.train()
      loss_record = []
      realloss = []
      for x, y in train_loader:
        optimizer.zero_grad()               # Set gradient to zero.
        x, y = x.to(device), y.to(device)   # Move your data to device. 
        pred = model(x)
        
        #loss = criterion(pred, y)
        l2_norm = sum((p.pow(2)/(1+p.pow(2))).sum()for p in model.parameters())        ##上課的l2 norm resize 過的不知道加個log會不會更好
        loss = criterion(pred, y)+config['regulizer']*l2_norm+config['loss2']*loss2(pred, y)
        realloss2 = criterion2(pred, y)
        #print(loss)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        
        loss_record.append(loss.detach().item())
        realloss.append(realloss2.item())

      mean_train_loss = sum(loss_record)/len(loss_record)         ###compute CV loss
      writer.add_scalar('Loss/train', mean_train_loss, step)
      reloss = sum(realloss)/len(realloss)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, L1 loss: {reloss:.4f}')
      if mean_train_loss < best_loss:
          best_loss = mean_train_loss
          torch.save(model.state_dict(), config['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
          

## config

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'n_fold': 5,   # validation_size = train_size * valid_ratio
    'n_epochs': 30000,     # Number of epochs.            
    'batch_size': 64, 
    'learning_rate': 1e-3,
    'regulizer': 1e-4,  
    'loss2'  :0,          
    'early_stop': 200,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}


In [ ]:
same_seed(config['seed'])

#imp_median.fit(test)
#test_data = imp_median.transform(test)
train_data= feature
#train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

x_test =  test_data
print(x_test.shape)
test_dataset = COVID19Dataset(x_test)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

(6315, 29)


## Training and testing

In [ ]:



model = My_Model(input_dim=train_data.shape[1]-1).to(device) # put your model and data on the same computation device.
trainer(train_data, model, config, device)






Epoch [1/30000]: Train loss: 2.3199, Valid loss: 1.7596, L1 loss: 1.8373   0
Saving model with loss 1.760...
Epoch [2/30000]: Train loss: 1.8836, Valid loss: 1.7851, L1 loss: 1.8684   0
Epoch [3/30000]: Train loss: 1.8214, Valid loss: 1.6799, L1 loss: 1.7587   0
Saving model with loss 1.680...
Epoch [4/30000]: Train loss: 1.7447, Valid loss: 1.6644, L1 loss: 1.7501   2
Saving model with loss 1.664...
Epoch [5/30000]: Train loss: 1.7017, Valid loss: 1.7045, L1 loss: 1.7845   3
Epoch [6/30000]: Train loss: 1.6918, Valid loss: 1.6034, L1 loss: 1.6950   3
Saving model with loss 1.603...
Epoch [7/30000]: Train loss: 1.6594, Valid loss: 1.5700, L1 loss: 1.6558   5
Saving model with loss 1.570...
Epoch [8/30000]: Train loss: 1.6341, Valid loss: 1.5337, L1 loss: 1.6267   6
Saving model with loss 1.534...
Epoch [9/30000]: Train loss: 1.6291, Valid loss: 1.6003, L1 loss: 1.7024   7
Epoch [10/30000]: Train loss: 1.6075, Valid loss: 1.6042, L1 loss: 1.7086   7
Epoch [11/30000]: Train loss: 1.6021,

KeyboardInterrupt: ignored

In [ ]:

trainer2(train_data, model,20 ,config, device)

Epoch [1/30000]: Train loss: 1.3804, L1 loss: 1.5891
Saving model with loss 1.380...
Epoch [2/30000]: Train loss: 1.3779, L1 loss: 1.5863
Saving model with loss 1.378...
Epoch [3/30000]: Train loss: 1.3779, L1 loss: 1.5864
Saving model with loss 1.378...
Epoch [4/30000]: Train loss: 1.3730, L1 loss: 1.5818
Saving model with loss 1.373...
Epoch [5/30000]: Train loss: 1.3579, L1 loss: 1.5662
Saving model with loss 1.358...
Epoch [6/30000]: Train loss: 1.3700, L1 loss: 1.5788
Epoch [7/30000]: Train loss: 1.3621, L1 loss: 1.5708
Epoch [8/30000]: Train loss: 1.3732, L1 loss: 1.5824
Epoch [9/30000]: Train loss: 1.3634, L1 loss: 1.5714
Epoch [10/30000]: Train loss: 1.3671, L1 loss: 1.5754
Epoch [11/30000]: Train loss: 1.3588, L1 loss: 1.5670
Epoch [12/30000]: Train loss: 1.3614, L1 loss: 1.5697
Epoch [13/30000]: Train loss: 1.3639, L1 loss: 1.5727
Epoch [14/30000]: Train loss: 1.3606, L1 loss: 1.5698
Epoch [15/30000]: Train loss: 1.3673, L1 loss: 1.5755
Epoch [16/30000]: Train loss: 1.3645, L

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'Danceability'])
        for i, p in enumerate(preds):
            writer.writerow([i+17170, p])
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
print(preds)
save_pred(preds, 'sample_submission.csv')
from google.colab import files
files.download('sample_submission.csv')

100%|██████████| 99/99 [00:00<00:00, 865.20it/s]

[5.6246457 5.000408  3.269388  ... 2.8322568 5.326324  7.627432 ]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>